In [4]:
from matplotlib import pyplot as plt
import pandas as pd
import fastf1 
import optuna
import seaborn as sns
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
import sys
from sktime.distances import distance
sys.path.append('..')
##sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from telemetry.telemetry_data_preprocessing import telemetry_computations
from loader import load_telemetry, download_all_season
from telemetry.telemetry_data_preprocessing import TelemetryProcessing

In [5]:
# folder_path = '../data_telemetry'

# final_lap_data = pd.DataFrame()

# for filename in os.listdir(folder_path):
#     file_path = os.path.join(folder_path, filename)
#     if os.path.isfile(file_path):  
        
        
#         telemetry = pd.DataFrame(load_telemetry(f"{folder_path}/{filename}"))
        
#         tel = TelemetryProcessing(telemetry)
        
#         tel.normalize_drs()
#         tel.compute_accelerations()
#         tel.calculate_lap_progress()

#         single_lap_data = tel.get_single_lap_data()
        
        
#         final_lap_data = pd.concat([final_lap_data, single_lap_data], axis=0)
        
#         print(f"loaded and processed {filename} data...")
        
# final_lap_data

In [6]:
tel = load_telemetry('../data_telemetry/Bahrain.pkl')
tel = TelemetryProcessing(tel)
tel.compute_accelerations()

In [7]:
tel.data

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,...,Y,Z,DriverNumber,LapNumber,LonAcc,LatAcc,AbsLatAcc,AbsLonAcc,SumLatAcc,SumLonAcc
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,,0.155556,0 days 00:00:00,10076.374938,0.000000,1,16.0,False,...,3558.983052,-156.999885,1,1.0,0.00000,0.00000,0.00000,0.00000,840.76762,561.07970
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,,0.155556,0 days 00:00:00.110000,10130.000000,0.000000,1,16.0,False,...,3558.982418,-156.999881,1,1.0,0.00000,0.00000,0.00000,0.00000,840.76762,561.07970
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,,0.155556,0 days 00:00:00.185000,9993.499884,0.000000,1,16.0,False,...,3559.000000,-157.000000,1,1.0,0.00000,-0.00000,0.00000,0.00000,840.76762,561.07970
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,,0.155556,0 days 00:00:00.310000,9766.000000,0.000000,1,16.0,False,...,3559.060983,-157.000413,1,1.0,0.10502,-0.00000,0.00000,0.10502,840.76762,561.07970
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,,0.155556,0 days 00:00:00.445000,8956.001152,0.964284,1,16.0,False,...,3559.000000,-157.000000,1,1.0,0.20226,-0.45947,0.45947,0.20226,840.76762,561.07970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799719,2023-03-05 15:26:01.015,0 days 01:24:59.166000,2,260.545556,0 days 00:02:01.769000,4050.000000,55.000000,8,0.0,False,...,1166.915551,-159.000000,81,13.0,-0.02408,4.03056,4.03056,0.02408,781.10969,459.80696
799720,2023-03-05 15:26:01.065,0 days 01:24:59.216000,2,264.059435,0 days 00:02:01.819000,4041.964306,54.821429,8,0.0,False,...,1191.000000,-159.000000,81,13.0,-0.10113,4.83611,4.83611,0.10113,781.10969,459.80696
799721,2023-03-05 15:26:01.295,0 days 01:24:59.446000,2,280.223333,0 days 00:02:02.049000,4005.000000,54.000000,8,0.0,False,...,1238.625511,-159.000000,81,13.0,-0.09691,4.62245,4.62245,0.09691,781.10969,459.80696
799722,2023-03-05 15:26:01.305,0 days 01:24:59.456000,2,280.223333,0 days 00:02:02.059000,4003.214309,54.000000,8,0.0,False,...,1240.000000,-159.000000,81,13.0,0.00000,4.62245,4.62245,0.00000,781.10969,459.80696


In [8]:
tel.data
race_name = 'Bahrain'
race_year =  2023

metric = 'dtw'

session = fastf1.get_session(race_year, race_name, 'Q')
    
session.load(telemetry=True)


ref_singnal_df = pd.DataFrame()

for driver in tel.data['DriverNumber'].unique():
    
    q_lap = session.laps.pick_driver(driver).pick_fastest().get_telemetry()
    
    q_lap['DriverNumber'] = driver
    q_lap['LapNumber'] = 1.0
    
    q_lap = TelemetryProcessing(q_lap)
    q_lap.compute_accelerations()
    
    q_lap.data.drop(columns='LapNumber', inplace=True)
    
    ref_singnal_df = pd.concat([ref_singnal_df, q_lap.data], axis=0)

req         WARNING 	DEFAULT CACHE ENABLED! (153.79 MB) C:\Users\Patryk\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '55', '14', '63', '44', '18', '31', '27', '4', '77', '24', '22', '23', '2', '20', '81', '21', '10']
c:\Users\P

In [12]:
distance_records = []

#ref_singnal_df = ref_singnal_df.set_index('DriverNumber')

for driver, group in tel.data.groupby('DriverNumber'):

    ref_signal = ref_singnal_df.loc[driver]

    for lap in group['LapNumber'].unique():
        lap_df = group[group['LapNumber'] == lap]
        distances_lon = distance(np.array(ref_signal['LonAcc']), np.array(lap_df['LonAcc']), metric=metric)
        distances_lat = distance(np.array(ref_signal['LatAcc']), np.array(lap_df['LatAcc']), metric=metric)
        
        distance_records.append({
                'DriverNumber': driver,
                'LapNumber': lap,
                f'LonDistance{metric}': distances_lon,
                f'LatDistance{metric}': distances_lat
            })

    print(f'processed driver: {driver}')
        
dist_df = pd.DataFrame(distance_records)
tel.data = tel.data.merge(dist_df, on=['DriverNumber', 'LapNumber'], how='left')

tel.data

KeyboardInterrupt: 

In [ ]:
results_df = pd.DataFrame(tel.data[tel.data['DriverNumber'] == 1.0])
plt.figure(figsize=(15, 5))
plt.plot(results_df["LapNumber"], results_df["LonDistancedtw"], marker="o")

for _, row in results_df.iterrows():
    plt.annotate(str(row["lap"]), (row["lap"], row["dtw_distance"]), textcoords="offset points", xytext=(0, 5), ha='center', fontsize=8)

plt.axhline(results_df["dtw_distance"].mean(), color="gray", linestyle="--", label="średnia DTW")
plt.title(f"Podobieństwo stylu jazdy (DTW) opartego na lat_acc względem najlepszego okrążenia kwalifikacyjnegp")
plt.xlabel("Numer okrążenia")
plt.ylabel("DTW (mniejsza = bardziej podobne)")
plt.legend()
plt.grid(True)
plt.show()